# 作业

请通过继承`Data`类实现一个类，专门用于表示“机构-作者-论文”的网络。该网络包含“机构“、”作者“和”论文”三类节点，以及“作者-机构“和“作者-论文“两类边。对要实现的类的要求：1）用不同的属性存储不同节点的属性；2）用不同的属性存储不同的边（边没有属性）；3）逐一实现获取不同节点数量的方法。

In [9]:
class Data:
    
    def __init__(self, x=None, edge_index=None, edge_attr=None, y=None, **kwargs):
        r"""
        Args:
            x (Tensor, optional): 节点属性矩阵，大小为`[num_nodes, num_node_features]`
            edge_index (LongTensor, optional): 边索引矩阵，大小为`[2, num_edges]`，第0行为 尾节点，第1行为头节点，头指向尾
            edge_attr (Tensor, optional): 边属性矩阵，大小为`[num_edges, num_edge_features]`
            y (Tensor, optional): 节点或图的标签，任意大小(，其实也可以是边的标签)
        """
        self.x = x
        self.edge_index = edge_index
        self.edge_attr = edge_attr
        self.y = y
        for key, item in kwargs.items():
            if key == 'num_nodes':
                self.__num_nodes__ = item
            else:
                self[key] = item
                
class MyData(Data):
    def __init__(self, node_paper=None, node_institute=None, node_author=None,
                 edge_index_author_institute=None,edge_index_paper_author=None,y=None):  
        super(MyData,self).__init__(y=y)  
        self.node_paper = node_paper
        self.node_institute = node_institute
        self.node_author = node_author
        self.edge_index_author_institute = edge_index_author_institute
        self.edge_index_paper_author = edge_index_paper_author
    
    @property
    def node_paper_num(self):
        return self.node_paper.shape[0]
    
    @property
    def node_author_num(self):
        return self.node_author.shape[0]
    
    @property
    def node_institute_num(self):
        return self.node_institute.shape[0]

# 学习笔记

## 简单图论

### 图结构数据



#### 一、图的表示

**定义一（图）**：
    
节点集合，边集合

节点表示实体（entities ），边表示实体间的关系（relations）。

节点和边的信息可以为类别型和数值型

在图的计算任务中，我们认为，节点一定含有信息（至少含有节点的度的信息），边可能含有信息。

**定义二（图的邻接矩阵）**：

Aij不为0表示从节点i到节点j之间有边，为0表示没有变

无向图的邻接矩阵是对称的

无权图各条边的权重为1

有权图，其对应的邻接矩阵通常被记为$\mathbf{W} \in\{0,1\}^{N \times N}$，其中$\mathbf{W}_{i, j}=w_{ij}$表示从节点$v_i$到$v_j$的边的权重。若边不存在时，边的权重为$0$。
	


#### 二、图的属性

**定义三（节点的度，degree）**：

有向有权图，节点出度（out degree）等于从节点出发的边的权重之和，节点的入度（in degree）等于从连向节点的边的权重之和。

无向图是有向图的特殊情况，节点的出度与入度相等。

无权图是有权图的特殊情况，各边的权重为1，

节点$v_i$的度记为$d(v_i)$，入度记为$d_{in}(v_i)$，出度记为$d_{out}(v_i)$。

**定义四（邻接节点，neighbors）**：

$\mathcal{N(v_i)}$。

节点$v_i$的$k$跳远的邻接节点（neighbors with $k$-hop）指的是到节点$v_i$要走$k$步的节点（一个节点的$2$跳远的邻接节点包含了自身）。

**定义五（行走，walk）**：


在“行走”中，节点是允许重复的。


**定理六**：

- 有一图，其邻接矩阵为 $\mathbf{A}$, $\mathbf{A}^{n}$为邻接矩阵的$n$次方，那么$\mathbf{A}^{n}[i,j]$等于从节点$v_i$到节点$v_j$的长度为$n$的行走的个数。（也就是，以节点$v_i$为起点，节点$v_j$为终点，长度为$n$的节点访问方案的数量，节点访问中可以兜圈子重复访问一些节点）

**定义七（路径，path）**：

“路径”是节点不可重复的“行走”。

**定义八（子图，subgraph）**：

字面意思

**定义九（连通分量，connected component）**：

看英文connected component更好理解，形象理解为两个子图互不连通，为整图的联通分量



**定义十（连通图，connected graph）**：

- 当一个图只包含一个连通分量，即其自身，那么该图是一个连通图。

**定义十一（最短路径，shortest path）**：

字面意义

**定义十二（直径，diameter）**：

- 给定一个连通图$\mathcal{G}=\{\mathcal{V}, \mathcal{E}\}$，其直径为其所有节点对之间的**最短路径的最大值**，形式化定义为

$$
\operatorname{diameter}(\mathcal{G})=\max _{v_{s}, v_{t} \in \mathcal{V}} \min _{p \in \mathcal{P}_{s t}}|p|
$$

**定义十三（拉普拉斯矩阵，Laplacian Matrix）**：

- 给定一个图$\mathcal{G}=\{\mathcal{V}, \mathcal{E}\}$，其邻接矩阵为$A$，其拉普拉斯矩阵定义为$\mathbf{L=D-A}$，其中$\mathbf{D=diag(d(v_1), \cdots, d(v_N))}$。

**定义十四（对称归一化的拉普拉斯矩阵，Symmetric normalized Laplacian）**：

- 给定一个图$\mathcal{G}=\{\mathcal{V}, \mathcal{E}\}$，其邻接矩阵为$A$，其规范化的拉普拉斯矩阵定义为

$$
\mathbf{L=D^{-\frac{1}{2}}(D-A)D^{-\frac{1}{2}}=I-D^{-\frac{1}{2}}AD^{-\frac{1}{2}}}
$$

#### 三、图的种类

- **同质图**（Homogeneous Graph）：只有一种类型的节点和一种类型的边的图。
- **异质图**（Heterogeneous Graph）：存在多种类型的节点和多种类型的边的图。
  
- **二部图**（Bipartite Graphs）：节点分为两类，只有不同类的节点之间存在边。
  

#### 四、图结构数据上的机器学习

1. **节点预测**：预测节点的类别或某类属性的取值
   1. 例子：对是否是潜在客户分类、对游戏玩家的消费能力做预测
2. **边预测**：预测两个节点间是否存在链接
   1. 例子：Knowledge graph completion、好友推荐、商品推荐
3. **图的预测**：对不同的图进行分类或预测图的属性
   1. 例子：分子属性预测
4. **节点聚类**：检测节点是否形成一个社区
   1. 例子：社交圈检测
5. **其他任务**
   1. **图生成**：例如药物发现
   2. **图演变**：例如物理模拟
   3. ……

#### 五、应用神经网络于图面临的挑战

在学习了简单的图论知识，我们再来回顾应用神经网络于图面临的挑战。

过去的深度学习应用中，我们主要接触的数据形式主要是这四种：**矩阵、张量、序列（sequence）和时间序列（time series）**，**它们都是规则的结构化的数据。然而图数据是非规则的非结构化的**，它具有以下的特点：

1. **任意的大小和复杂的拓扑结构；**
2. **没有固定的节点排序或参考点；**
3. **通常是动态的，并具有多模态的特征；**
4. **图的信息并非只蕴含在节点信息和边的信息中，图的信息还包括了图的拓扑结构。**



以往的深度学习技术是为规则且结构化的数据设计的，无法直接用于图数据。应用于图数据的神经网络，要求

- **适用于不同度的节点**；
- **节点表征的计算与邻接节点的排序无关**；
- **不但能够根据节点信息、邻接节点的信息和边的信息计算节点表征，还能根据图拓扑结构计算节点表征**。下



#### 参考资料

- [Chapter 2 - Foundations of Graphs, Deep Learning on Graphs](https://cse.msu.edu/~mayao4/dlg_book/chapters/chapter2.pdf)




## pyG环境配置

环境：阿里云gpu计算型（阿里云gpu虚拟型不能安装显卡驱动，计算型会自动装）



使用`nvidia-smi`查看显卡驱动是否正确安装

安装1.8.1版本的pytorch和11.1版本的cudatoolkit

`conda install pytorch torchvision torchaudio cudatoolkit=11.1 -c pytorch -c nvidia`

确认是否正确安装，正确的安装应出现下方的结果

```
$ python -c "import torch; print(torch.__version__)"
# 1.8.1
$ python -c "import torch; print(torch.version.cuda)"
# 11.1
```

安装对应版本的PyG
```
pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.8.0+cu111.html
pip install torch-geometric
```

注：在阿里云上安装`pip install torch-geometric`这一步会出错，使用`pip install pytest-runner`之后再安装torch-geometric就成功了

## 数据集测试

### Data

In [26]:
from torch_geometric.datasets import KarateClub

dataset = KarateClub()
data = dataset[0]  # Get the first graph object.
print(data)
print('==============================================================')

# 获取图的一些信息
print(f'Number of nodes: {data.num_nodes}') # 节点数量
print(f'Number of edges: {data.num_edges}') # 边数量
print(f'Number of node features: {data.num_node_features}') # 节点属性的维度
print(f'Number of node features: {data.num_features}') # 同样是节点属性的维度
print(f'Number of edge features: {data.num_edge_features}') # 边属性的维度
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}') # 平均节点度
print(f'if edge indices are ordered and do not contain duplicate entries.: {data.is_coalesced()}') # 是否边是有序的同时不含有重复的边
print(f'Number of training nodes: {data.train_mask.sum()}') # 用作训练集的节点
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}') # 用作训练集的节点数占比
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}') # 此图是否包含孤立的节点
print(f'Contains self-loops: {data.contains_self_loops()}')  # 此图是否包含自环的边
print(f'Is undirected: {data.is_undirected()}')  # 此图是否是无向图

Data(edge_index=[2, 156], train_mask=[34], x=[34, 34], y=[34])
Number of nodes: 34
Number of edges: 156
Number of node features: 34
Number of node features: 34
Number of edge features: 0
Average node degree: 4.59
if edge indices are ordered and do not contain duplicate entries.: True
Number of training nodes: 4
Training node label rate: 0.12
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


### Dataset

In [31]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
#Planetoid.url='https://gitee.com/rongqinchen/planetoid/raw/master/data/'
#Planetoid.url='https://gitee.com/kimiyoung/planetoid/daata/'
dataset = Planetoid(root='./dataset/Cora', name='Cora',transform=NormalizeFeatures())
# Cora()

len(dataset)
# 1

dataset.num_classes
# 7

dataset.num_node_features

Processing...
Done!


1433